In [24]:
import pandas as pd

data = pd.read_csv('/content/train.csv')

print(data.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64


In [25]:
평균 = data['Age'].mean()
최빈값 = data['Embarked'].mode()[0]

data['Age'].fillna(평균, inplace=True)
data['Embarked'].fillna(최빈값, inplace=True)

print(data.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


<ipython-input-25-787fbf6e59b8>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(평균, inplace=True)
<ipython-input-25-787fbf6e59b8>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method

In [26]:
import tensorflow as tf

정답 = data.pop('Survived')



In [27]:

ds = tf.data.Dataset.from_tensor_slices(((dict(data)), 정답))
print(ds)

<_TensorSliceDataset element_spec=({'PassengerId': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Pclass': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Name': TensorSpec(shape=(), dtype=tf.string, name=None), 'Sex': TensorSpec(shape=(), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(), dtype=tf.float64, name=None), 'SibSp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Parch': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Ticket': TensorSpec(shape=(), dtype=tf.string, name=None), 'Fare': TensorSpec(shape=(), dtype=tf.float64, name=None), 'Embarked': TensorSpec(shape=(), dtype=tf.string, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [10]:
feature_columns = []
# 숫자
feature_columns.append(tf.feature_column.numeric_column('Fare'))
feature_columns.append(tf.feature_column.numeric_column('SibSp'))
feature_columns.append(tf.feature_column.numeric_column('Parch'))

#뭉뚱 카테고리
Age= tf.feature_column.numeric_column('Age')
Age_bucket = tf.feature_column.bucketized_column(Age, boundaries=[10, 20, 30, 40, 50, 60, 70, 80])
feature_columns.append(Age_bucket)

# 카테고리
vocab = data['Sex'].unique()
Sex = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Sex', vocab))
feature_columns.append(Sex)

vocab = data['Embarked'].unique()
Embarked = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Embarked', vocab))
feature_columns.append(Embarked)

vocab = data['Pclass'].unique()
Pclass = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Pclass', vocab))
feature_columns.append(Pclass)


AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'DenseFeatures'

In [7]:
#embedding (카테고리가 너무 많아)
vocab = data['Ticket'].unique()
Ticket = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_vocabulary_list('Ticket', vocab), dimension = 9)
feature_columns.append(Ticket)


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [12]:
import tensorflow as tf
print(tf.__version__)


2.17.1


In [28]:
import tensorflow as tf

# Feature columns 정의
feature_columns = []

# 숫자형 feature columns 추가
feature_columns.append(tf.feature_column.numeric_column('Fare'))
feature_columns.append(tf.feature_column.numeric_column('SibSp'))
feature_columns.append(tf.feature_column.numeric_column('Parch'))

# Age에 대한 bucketized feature column 추가
Age = tf.feature_column.numeric_column('Age')
Age_bucket = tf.feature_column.bucketized_column(Age, boundaries=[10, 20, 30, 40, 50, 60, 70, 80])
feature_columns.append(Age_bucket)

# 범주형 feature columns 추가 (Sex, Embarked, Pclass)
vocab = data['Sex'].unique()
Sex = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Sex', vocab))
feature_columns.append(Sex)

vocab = data['Embarked'].unique()
Embarked = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Embarked', vocab))
feature_columns.append(Embarked)

vocab = data['Pclass'].unique()
Pclass = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Pclass', vocab))
feature_columns.append(Pclass)

# Embedding feature column 추가 (Ticket)
vocab = data['Ticket'].unique()
Ticket = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_vocabulary_list('Ticket', vocab), dimension=9)
feature_columns.append(Ticket)

# Feature columns을 Dense 레이어로 변환하기 위한 방법
# 모델 입력 정의
input_layer = tf.keras.layers.Input(shape=(len(feature_columns),))

# 각 feature_column에 대해 Dense 레이어로 변환
x = tf.keras.layers.Dense(128, activation='relu')(input_layer)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)

# 출력층 정의 (이진 분류를 예시로 설정)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# 모델 정의
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 구조 확인
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,521 (45.00 KB)

 Trainable params: 11,521 (45.00 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 전체 데이터셋을 TensorFlow Dataset으로 변환
ds = tf.data.Dataset.from_tensor_slices((dict(data), 정답))

# 데이터셋을 훈련 데이터와 검증 데이터로 나누기
train_data, val_data = train_test_split(list(ds), test_size=0.2, random_state=42)

# tf.data.Dataset으로 변환
train_dataset = tf.data.Dataset.from_generator(
    lambda: iter(train_data),
    output_signature=(
        {
            'PassengerId': tf.TensorSpec(shape=(), dtype=tf.int64),
            'Pclass': tf.TensorSpec(shape=(), dtype=tf.int64),
            'Name': tf.TensorSpec(shape=(), dtype=tf.string),
            'Sex': tf.TensorSpec(shape=(), dtype=tf.string),
            'Age': tf.TensorSpec(shape=(), dtype=tf.float64),
            'SibSp': tf.TensorSpec(shape=(), dtype=tf.int64),
            'Parch': tf.TensorSpec(shape=(), dtype=tf.int64),
            'Ticket': tf.TensorSpec(shape=(), dtype=tf.string),
            'Fare': tf.TensorSpec(shape=(), dtype=tf.float64),
            'Embarked': tf.TensorSpec(shape=(), dtype=tf.string)
        },
        tf.TensorSpec(shape=(), dtype=tf.float32)  # 정답 데이터 (Survived)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: iter(val_data),
    output_signature=(
        {
            'PassengerId': tf.TensorSpec(shape=(), dtype=tf.int64),
            'Pclass': tf.TensorSpec(shape=(), dtype=tf.int64),
            'Name': tf.TensorSpec(shape=(), dtype=tf.string),
            'Sex': tf.TensorSpec(shape=(), dtype=tf.string),
            'Age': tf.TensorSpec(shape=(), dtype=tf.float64),
            'SibSp': tf.TensorSpec(shape=(), dtype=tf.int64),
            'Parch': tf.TensorSpec(shape=(), dtype=tf.int64),
            'Ticket': tf.TensorSpec(shape=(), dtype=tf.string),
            'Fare': tf.TensorSpec(shape=(), dtype=tf.float64),
            'Embarked': tf.TensorSpec(shape=(), dtype=tf.string)
        },
        tf.TensorSpec(shape=(), dtype=tf.float32)  # 정답 데이터 (Survived)
    )
)

# 데이터를 float32로 변환하고, 배치 처리
train_dataset = train_dataset.map(lambda features, label: (features, tf.cast(label, tf.float32)))
val_dataset = val_dataset.map(lambda features, label: (features, tf.cast(label, tf.float32)))

train_dataset = train_dataset.batch(32)
val_dataset = val_dataset.batch(32)

# 모델 학습
model.fit(train_dataset, epochs=10, validation_data=val_dataset)


Epoch 1/10


ValueError: Missing data for input "input_layer_5". You passed a data dictionary with keys ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']. Expected the following keys: ['input_layer_5']